In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from mne import Epochs, pick_types, events_from_annotations
from mne.io.edf import read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from sklearn.model_selection import cross_val_score, KFold

import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, cohen_kappa_score
from sklearn.model_selection import KFold
from kan.KAN import KAN 
import mne
from mne.datasets import eegbci  
from mne import EpochsArray, pick_types, events_from_annotations
from mne.decoding import CSP  
from torch.nn import Sequential, Linear, ReLU, CrossEntropyLoss
from torch.optim import LBFGS 
from kan.KAN import KAN 
from sklearn.preprocessing import LabelEncoder
torch.set_default_dtype(torch.float64)
from mrmr import mrmr_classif
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  

In [2]:
# avoid classification of evoked responses by using epochs that start 1s after
# cue onset.
def calculate_metrics(output, target):
    preds = output.argmax(dim=1)
    accuracy = (preds == target).sum().item() / len(target)
    f1 = f1_score(target.cpu(), preds.cpu(), average='weighted') 
    kappa = cohen_kappa_score(target.cpu(), preds.cpu())
    return accuracy, f1, kappa

def my_loss_fn(output, target):
    loss = CrossEntropyLoss()(output, target)
    return loss.double()

def iteration_subjects(subjects, runs,event_id):
    tmin, tmax = -1., 3.
    epochs_data=[]
    for subject in subjects:
        for data in eegbci.load_data(subject, runs):
            raw=read_raw_edf(data, preload=True)
            picks = pick_types(
                            raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')

            raw.filter(7., 35., method='iir', picks=picks)
            events, _ = events_from_annotations(raw, event_id=event_id)
            epochs = Epochs(
                        raw,
                        events,
                        event_id,
                        tmin,
                        tmax,
                        proj=True,
                        picks=picks,
                        baseline=None,
                        preload=True,
                        verbose=False)
            if event_id['T1']>2:
                epochs.event_id = {'T3': 3, 'T4': 4}
            epochs_data.append(epochs)
    return epochs_data


def data_EEG(train=True, multiclass=False):
    #6, 10, 14 	Motor imagery: hands vs feet
    #4, 8, 12   Motor imagery: left vs right hand

    event_id = dict(T1=1, T2=2)
    subjects = [1]  # [77, 16, 62, 9, 46, 30, 24, 84, 56, 95]
    if train:
        runs = [4, 8]  # motor imagery: left - right 
    else:
        runs = [12]
    epochs_data = iteration_subjects(subjects, runs,event_id)
    if multiclass:
        event_id = dict(T1=3, T2=4)
        if train:
            runs = [6, 10]  # motor imagery: left - right 
        else:
            runs = [14]
        epochs_data.extend(iteration_subjects(subjects, runs,event_id))
        return mne.concatenate_epochs(epochs_data)[['T1', 'T2', 'T4']]

    
    return mne.concatenate_epochs(epochs_data)


In [3]:
def creating_models(input_shape, output_shape):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    kan_model = KAN(width=[input_shape, 5, 5, output_shape], grid=100, k=10, seed=0, device=device)             
    for param in kan_model.parameters():
        param.data = param.data.double()
    mlp_model = Sequential(
      Linear(input_shape, 8).double(),  
      ReLU(),
      Linear(8, output_shape).double()  # 2 output classes 
      )

    for param in mlp_model.parameters():
        param.data = param.data.double()
        
    return kan_model,mlp_model


def train_model(model, optimizer, X_train, y_train):
    optimizer.zero_grad()
    output = model(X_train)
    loss = CrossEntropyLoss()(output, y_train) 
    loss.backward()
    optimizer.step()




import torch 
from torch.utils.data import Dataset

class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).double() # Or .float() if single precision is sufficient
        self.y = torch.from_numpy(y).long()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

#Raw - Holdout

In [4]:
df= data_EEG().to_data_frame().drop(columns=['time', 'epoch'])
# Data Conversion (NumPy to PyTorch)
selected_features = mrmr_classif(X=df.drop(columns=['condition']), y=df.condition, K=10)
df_features = df[selected_features+['condition']]
X = df.drop(columns=['condition']).values
y = df.condition.values
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train = torch.from_numpy(X_train).double()
X_test  = torch.from_numpy(X_test).double()
y_train  = torch.from_numpy(y_train).long()
y_test  = torch.from_numpy(y_test).long()



Download complete in 23s (5.0 MB)
Extracting EDF parameters from C:\Users\JARS\datasets\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 35 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 7.00, 35.00 Hz: -6.02, -6.02 dB

Used Annotations descriptions: ['T1', 'T2']
Extracting EDF parameters from C:\Users\JARS\datasets\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 35 Hz

IIR filter parameters


100%|██████████| 10/10 [00:00<00:00, 33.60it/s]


In [5]:
# ... (Rest of the code: accuracy, training loop)
kan_model, mlp_model  = creating_models(X.shape[1], len(np.unique(y)))
kan_optimizer = LBFGS(kan_model.parameters()) 
mlp_optimizer = LBFGS(mlp_model.parameters())  

epochs = 50
KAN_metrics_raw_hld= {}
MLP_metrics_raw_hld= {}
KAN_metrics_raw_hld['train']= []
KAN_metrics_raw_hld['test']= []
MLP_metrics_raw_hld['train']= []
MLP_metrics_raw_hld['test']= []

for epoch in range(epochs):
    ### KAN Training
    def kan_closure():  
        kan_optimizer.zero_grad()
        output = kan_model(X_train)
        loss =  CrossEntropyLoss()(output, y_train)
        loss.backward()
        return loss
    kan_optimizer.step(kan_closure)

    def mlp_closure():
        mlp_optimizer.zero_grad()
        output = mlp_model(X_train)
        loss = CrossEntropyLoss()(output, y_train)  # Using standard loss
        loss.backward()
        return loss
    mlp_optimizer.step(mlp_closure)

    kan_train_acc, kan_train_f1, kan_train_kappa = calculate_metrics(kan_model(X_train), y_train)
    kan_test_acc, kan_test_f1, kan_test_kappa = calculate_metrics(kan_model(X_test), y_test) 

    mlp_train_acc, mlp_train_f1, mlp_train_kappa = calculate_metrics(mlp_model(X_train), y_train)
    mlp_test_acc, mlp_test_f1, mlp_test_kappa = calculate_metrics(mlp_model(X_test), y_test)
    print(f'Epoch {epoch+1}/{epochs} | '
          f'KAN Train Acc: {kan_train_acc:.3f} | KAN F1: {kan_train_f1:.3f} | KAN Kappa: {kan_train_kappa:.3f} | '
          f'KAN Test Acc: {kan_test_acc:.3f} | KAN Test F1: {kan_test_f1:.3f} | KAN Test Kappa: {kan_test_kappa:.3f} | ')
    
    print(f'Epoch {epoch+1}/{epochs} | '
          f'MLP Train Acc: {mlp_train_acc:.3f} | MLP F1: {mlp_train_f1:.3f} | MLP Kappa: {mlp_train_kappa:.3f} | '
          f'MLP Test Acc: {mlp_test_acc:.3f} | MLP Test F1: {mlp_test_f1:.3f} | MLP Test Kappa: {mlp_test_kappa:.3f} | ')
    
    KAN_metrics_raw_hld['train'].append([kan_train_acc, kan_train_f1, kan_train_kappa])
    KAN_metrics_raw_hld['test'].append([kan_test_acc, kan_test_f1, kan_test_kappa ])
    MLP_metrics_raw_hld['train'].append([mlp_train_acc, mlp_train_f1, mlp_train_kappa])
    MLP_metrics_raw_hld['test'].append([mlp_test_acc, mlp_test_f1, mlp_test_kappa])

   

Epoch 1/50 | KAN Train Acc: 0.535 | KAN F1: 0.376 | KAN Kappa: 0.001 | KAN Test Acc: 0.529 | KAN Test F1: 0.368 | KAN Test Kappa: -0.000 | 
Epoch 1/50 | MLP Train Acc: 0.520 | MLP F1: 0.520 | MLP Kappa: 0.037 | MLP Test Acc: 0.517 | MLP Test F1: 0.517 | MLP Test Kappa: 0.030 | 
Epoch 2/50 | KAN Train Acc: 0.537 | KAN F1: 0.420 | KAN Kappa: 0.012 | KAN Test Acc: 0.525 | KAN Test F1: 0.409 | KAN Test Kappa: -0.000 | 
Epoch 2/50 | MLP Train Acc: 0.544 | MLP F1: 0.485 | MLP Kappa: 0.042 | MLP Test Acc: 0.536 | MLP Test F1: 0.478 | MLP Test Kappa: 0.035 | 
Epoch 3/50 | KAN Train Acc: 0.538 | KAN F1: 0.391 | KAN Kappa: 0.008 | KAN Test Acc: 0.527 | KAN Test F1: 0.376 | KAN Test Kappa: -0.002 | 
Epoch 3/50 | MLP Train Acc: 0.554 | MLP F1: 0.551 | MLP Kappa: 0.096 | MLP Test Acc: 0.540 | MLP Test F1: 0.537 | MLP Test Kappa: 0.071 | 
Epoch 4/50 | KAN Train Acc: 0.546 | KAN F1: 0.433 | KAN Kappa: 0.032 | KAN Test Acc: 0.525 | KAN Test F1: 0.409 | KAN Test Kappa: -0.002 | 
Epoch 4/50 | MLP Train 

In [ ]:
df= data_EEG(multiclass=True).to_data_frame().drop(columns=['time', 'epoch'])
# Data Conversion (NumPy to PyTorch)
selected_features = mrmr_classif(X=df.drop(columns=['condition']), y=df.condition, K=10)
df_features = df[selected_features+['condition']]
X = df.drop(columns=['condition']).values
y = df.condition.values
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train = torch.from_numpy(X_train).double()
X_test  = torch.from_numpy(X_test).double()
y_train  = torch.from_numpy(y_train).long()
y_test  = torch.from_numpy(y_test).long()

Extracting EDF parameters from C:\Users\JARS\datasets\MNE-eegbci-data\files\eegmmidb\1.0.0\S010\S010R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 35 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 7.00, 35.00 Hz: -6.02, -6.02 dB

Used Annotations descriptions: ['T1', 'T2']
Extracting EDF parameters from C:\Users\JARS\datasets\MNE-eegbci-data\files\eegmmidb\1.0.0\S010\S010R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 35 Hz

IIR filter parameters
---------------------
Butterworth 





100%|██████████| 10/10 [00:00<00:00, 26.46it/s]


In [ ]:
# ... (Rest of the code: accuracy, training loop)
kan_model, mlp_model  = creating_models(X.shape[1], len(np.unique(y)))
kan_optimizer = LBFGS(kan_model.parameters()) 
mlp_optimizer = LBFGS(mlp_model.parameters())  

epochs = 50
KAN_metrics_raw_hld_mc= {}
MLP_metrics_raw_hld_mc= {}
KAN_metrics_raw_hld_mc['train']= []
KAN_metrics_raw_hld_mc['test']= []
MLP_metrics_raw_hld_mc['train']= []
MLP_metrics_raw_hld_mc['test']= []

for epoch in range(epochs):
    ### KAN Training
    def kan_closure():  
        kan_optimizer.zero_grad()
        output = kan_model(X_train)
        loss =  CrossEntropyLoss()(output, y_train)
        loss.backward()
        return loss
    kan_optimizer.step(kan_closure)

    def mlp_closure():
        mlp_optimizer.zero_grad()
        output = mlp_model(X_train)
        loss = CrossEntropyLoss()(output, y_train)  # Using standard loss
        loss.backward()
        return loss
    mlp_optimizer.step(mlp_closure)

    kan_train_acc, kan_train_f1, kan_train_kappa = calculate_metrics(kan_model(X_train), y_train)
    kan_test_acc, kan_test_f1, kan_test_kappa = calculate_metrics(kan_model(X_test), y_test) 

    mlp_train_acc, mlp_train_f1, mlp_train_kappa = calculate_metrics(mlp_model(X_train), y_train)
    mlp_test_acc, mlp_test_f1, mlp_test_kappa = calculate_metrics(mlp_model(X_test), y_test)
    print(f'Epoch {epoch+1}/{epochs} | '
          f'KAN Train Acc: {kan_train_acc:.3f} | KAN F1: {kan_train_f1:.3f} | KAN Kappa: {kan_train_kappa:.3f} | '
          f'KAN Test Acc: {kan_test_acc:.3f} | KAN Test F1: {kan_test_f1:.3f} | KAN Test Kappa: {kan_test_kappa:.3f} | ')
    
    print(f'Epoch {epoch+1}/{epochs} | '
          f'MLP Train Acc: {mlp_train_acc:.3f} | MLP F1: {mlp_train_f1:.3f} | MLP Kappa: {mlp_train_kappa:.3f} | '
          f'MLP Test Acc: {mlp_test_acc:.3f} | MLP Test F1: {mlp_test_f1:.3f} | MLP Test Kappa: {mlp_test_kappa:.3f} | ')
    
    KAN_metrics_raw_hld_mc['train'].append([kan_train_acc, kan_train_f1, kan_train_kappa])
    KAN_metrics_raw_hld_mc['test'].append([kan_test_acc, kan_test_f1, kan_test_kappa ])
    MLP_metrics_raw_hld_mc['train'].append([mlp_train_acc, mlp_train_f1, mlp_train_kappa])
    MLP_metrics_raw_hld_mc['test'].append([mlp_test_acc, mlp_test_f1, mlp_test_kappa])

   

Epoch 1/50 | KAN Train Acc: 0.335 | KAN F1: 0.275 | KAN Kappa: -0.006 | KAN Test Acc: 0.328 | KAN Test F1: 0.265 | KAN Test Kappa: -0.009 | 
Epoch 1/50 | MLP Train Acc: 0.364 | MLP F1: 0.276 | MLP Kappa: 0.018 | MLP Test Acc: 0.354 | MLP Test F1: 0.265 | MLP Test Kappa: 0.011 | 
Epoch 2/50 | KAN Train Acc: 0.360 | KAN F1: 0.252 | KAN Kappa: 0.009 | KAN Test Acc: 0.351 | KAN Test F1: 0.238 | KAN Test Kappa: 0.004 | 
Epoch 2/50 | MLP Train Acc: 0.382 | MLP F1: 0.306 | MLP Kappa: 0.049 | MLP Test Acc: 0.365 | MLP Test F1: 0.287 | MLP Test Kappa: 0.031 | 
Epoch 3/50 | KAN Train Acc: 0.362 | KAN F1: 0.260 | KAN Kappa: 0.013 | KAN Test Acc: 0.356 | KAN Test F1: 0.251 | KAN Test Kappa: 0.012 | 
Epoch 3/50 | MLP Train Acc: 0.390 | MLP F1: 0.316 | MLP Kappa: 0.062 | MLP Test Acc: 0.371 | MLP Test F1: 0.296 | MLP Test Kappa: 0.041 | 
Epoch 4/50 | KAN Train Acc: 0.331 | KAN F1: 0.326 | KAN Kappa: -0.010 | KAN Test Acc: 0.330 | KAN Test F1: 0.324 | KAN Test Kappa: -0.008 | 
Epoch 4/50 | MLP Train 

In [ ]:
df= data_EEG(multiclass=True).to_data_frame().drop(columns=['time', 'epoch'])
# Data Conversion (NumPy to PyTorch)
selected_features = mrmr_classif(X=df.drop(columns=['condition']), y=df.condition, K=10)
df_features = df[selected_features+['condition']]
X = df_features.drop(columns=['condition']).values
y = df_features.condition.values
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train = torch.from_numpy(X_train).double()
X_test  = torch.from_numpy(X_test).double()
y_train  = torch.from_numpy(y_train).long()
y_test  = torch.from_numpy(y_test).long()

#CSP Features

In [ ]:
X, y = data_EEG()
# Data Conversion (NumPy to PyTorch)
X = np.reshape(X, [np.shape(X)[0]*np.shape(X)[1], np.shape(X)[2]])
y = np.reshape(y, [np.shape(y)[0]*np.shape(y)[1]])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Convert to PyTorch tensors
X_train = torch.from_numpy(X_train).double() 
X_test = torch.from_numpy(X_test).double()
y_train = torch.from_numpy(y_train).long()  
y_test = torch.from_numpy(y_test).long()





# --- KAN Model --- 
model = KAN(width=[X_train.shape[1], 2],  # Input features, 2 output classes
            grid=5,          
            k=3)             

# Ensure double-precision tensors
for param in model.parameters():
    param.data = param.data.double()

optimizer = LBFGS(model.parameters()) 



# --- MLP Model --- 
mlp_model = Sequential(
      Linear(X_train.shape[1], 8).double(),  # Input features 
      ReLU(),
      Linear(8, 2).double()  # 2 output classes 
)

# Ensure double-precision tensors
for param in mlp_model.parameters():
    param.data = param.data.double()

mlp_optimizer = LBFGS(mlp_model.parameters())  

# ... (Rest of the code: accuracy, training loop)
epochs = 20

for epoch in range(epochs):
    ### KAN Training
    def kan_closure():  
        optimizer.zero_grad()
        output = model(X_train)
        loss =  CrossEntropyLoss()(output, y_train)
        loss.backward()
        return loss
    optimizer.step(kan_closure)

    ### MLP Training
    def mlp_closure():
        mlp_optimizer.zero_grad()
        output = mlp_model(X_train)
        loss = CrossEntropyLoss()(output, y_train)  # Using standard loss
        loss.backward()
        return loss
    mlp_optimizer.step(mlp_closure)

    kan_train_acc, kan_train_f1, kan_train_kappa = calculate_metrics(model(X_train), y_train)
    kan_test_acc, kan_test_f1, kan_test_kappa = calculate_metrics(model(X_test), y_test) 

    mlp_train_acc, mlp_train_f1, mlp_train_kappa = calculate_metrics(mlp_model(X_train), y_train)
    mlp_test_acc, mlp_test_f1, mlp_test_kappa = calculate_metrics(mlp_model(X_test), y_test)


    print(f'Epoch {epoch+1}/{epochs} | '
          f'KAN Train Acc: {kan_train_acc:.3f} | KAN F1: {kan_train_f1:.3f} | KAN Kappa: {kan_train_kappa:.3f} | '
          f'MLP Train Acc: {mlp_train_acc:.3f} | MLP F1: {mlp_train_f1:.3f} | MLP Kappa: {mlp_train_kappa:.3f}') 

#RAW - CV

In [ ]:
# ... (Rest of the code: accuracy, training loop)
epochs = 50
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # 5-fold cross-validation

from mrmr import mrmr_classif

df= data_EEG(raw_data=True)
# Data Conversion (NumPy to PyTorch)
selected_features = mrmr_classif(X=df.drop(columns=['events_id']), y=df.events_id, K=10)
df_features = df[selected_features+['events_id']]

# Data Conversion (NumPy to PyTorch)
X = df_features.drop(columns=['events_id']).values
y = df_features.events_id.values

X = torch.from_numpy(X).double()   # Convert to PyTorch Tensor 
y = torch.from_numpy(y).long()  # Convert to PyTorch tensor 



optimizer = LBFGS(model.parameters()) 



# --- MLP Model --- 

mlp_optimizer = LBFGS(mlp_model.parameters())  

for fold, (train_index, test_index) in enumerate(kf.split(X)):
    # Prepare data for this fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


# ... (Rest of the code: accuracy, training loop)
    epochs = 20

    for epoch in range(epochs):
        ### KAN Training
        def kan_closure():  
            optimizer.zero_grad()
            output = model(X_train)
            loss =  CrossEntropyLoss()(output, y_train)
            loss.backward()
            return loss
        optimizer.step(kan_closure)

        ### MLP Training
        def mlp_closure():
            mlp_optimizer.zero_grad()
            output = mlp_model(X_train)
            loss = CrossEntropyLoss()(output, y_train)  # Using standard loss
            loss.backward()
            return loss
        mlp_optimizer.step(mlp_closure)

        kan_train_acc, kan_train_f1, kan_train_kappa = calculate_metrics(model(X_train), y_train)
        kan_test_acc, kan_test_f1, kan_test_kappa = calculate_metrics(model(X_test), y_test) 

        mlp_train_acc, mlp_train_f1, mlp_train_kappa = calculate_metrics(mlp_model(X_train), y_train)
        mlp_test_acc, mlp_test_f1, mlp_test_kappa = calculate_metrics(mlp_model(X_test), y_test)

        print(f'KFOLD {fold}')
        print(f'Epoch {epoch+1}/{epochs} | '
            f'KAN Train Acc: {kan_train_acc:.3f} | KAN F1: {kan_train_f1:.3f} | KAN Kappa: {kan_train_kappa:.3f} | '
            f'MLP Train Acc: {mlp_train_acc:.3f} | MLP F1: {mlp_train_f1:.3f} | MLP Kappa: {mlp_train_kappa:.3f}') 

#CSP - CV

In [ ]:


# ... (Rest of the code: accuracy, training loop)
epochs = 50
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # 5-fold cross-validation

X, y = data_EEG()
X = np.reshape(X, [np.shape(X)[0]*np.shape(X)[1], np.shape(X)[2]])
y = np.reshape(y, [np.shape(y)[0]*np.shape(y)[1]])

X = torch.from_numpy(X).double()   # Convert to PyTorch Tensor 
y = torch.from_numpy(y).long()  # Convert to PyTorch tensor 

model = KAN(width=[X.shape[1], 2],  # Input features, 2 output classes
            grid=5,          
            k=3)             

# Ensure double-precision tensors
for param in model.parameters():
    param.data = param.data.double()

optimizer = LBFGS(model.parameters()) 



# --- MLP Model --- 
mlp_model = Sequential(
      Linear(X.shape[1], 8).double(),  # Input features 
      ReLU(),
      Linear(8, 2).double()  # 2 output classes 
)

# Ensure double-precision tensors
for param in mlp_model.parameters():
    param.data = param.data.double()

mlp_optimizer = LBFGS(mlp_model.parameters())  

for fold, (train_index, test_index) in enumerate(kf.split(X)):
    # Prepare data for this fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = KAN(width=[X_train.shape[1], 2],  # Input features, 2 output classes
            grid=5,          
            k=3)             

    # Ensure double-precision tensors
    for param in model.parameters():
        param.data = param.data.double()

    optimizer = LBFGS(model.parameters()) 



    # --- MLP Model --- 
    mlp_model = Sequential(
        Linear(X_train.shape[1], 8).double(),  # Input features 
        ReLU(),
        Linear(8, 2).double()  # 2 output classes 
    )

    # Ensure double-precision tensors
    for param in mlp_model.parameters():
        param.data = param.data.double()

    mlp_optimizer = LBFGS(mlp_model.parameters())  

# ... (Rest of the code: accuracy, training loop)
    epochs = 20

    for epoch in range(epochs):
        ### KAN Training
        def kan_closure():  
            optimizer.zero_grad()
            output = model(X_train)
            loss =  CrossEntropyLoss()(output, y_train)
            loss.backward()
            return loss
        optimizer.step(kan_closure)

        ### MLP Training
        def mlp_closure():
            mlp_optimizer.zero_grad()
            output = mlp_model(X_train)
            loss = CrossEntropyLoss()(output, y_train)  # Using standard loss
            loss.backward()
            return loss
        mlp_optimizer.step(mlp_closure)

        kan_train_acc, kan_train_f1, kan_train_kappa = calculate_metrics(model(X_train), y_train)
        kan_test_acc, kan_test_f1, kan_test_kappa = calculate_metrics(model(X_test), y_test) 

        mlp_train_acc, mlp_train_f1, mlp_train_kappa = calculate_metrics(mlp_model(X_train), y_train)
        mlp_test_acc, mlp_test_f1, mlp_test_kappa = calculate_metrics(mlp_model(X_test), y_test)

        print(f'KFOLD {fold}')
        print(f'Epoch {epoch+1}/{epochs} | '
            f'KAN Train Acc: {kan_train_acc:.3f} | KAN F1: {kan_train_f1:.3f} | KAN Kappa: {kan_train_kappa:.3f} | '
            f'MLP Train Acc: {mlp_train_acc:.3f} | MLP F1: {mlp_train_f1:.3f} | MLP Kappa: {mlp_train_kappa:.3f}') 

In [ ]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
import mne
import moabb
from moabb.datasets import BNCI2014_001
from moabb.evaluations import WithinSessionEvaluation
from moabb.paradigms import LeftRightImagery
from mne.decoding import CSP


moabb.set_log_level("info")
warnings.filterwarnings("ignore")
csp = CSP(n_components=4, reg='oas') 
dataset = BNCI2014_001()
dataset.subject_list = [1, 2, 3]
sessions = dataset.get_data(subjects=[1])

subject = 1
session_name = "0train"
run_name = "0"
raw = sessions[subject][session_name][run_name]
tmin, tmax = -1., 6.
events = mne.find_events(raw, stim_channel="stim")

event_id = dict(left=1, right=2, feet=3, tongue=4)
picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
epochs = Epochs(raw,
                events,
                event_id,
                tmin,
                tmax,
                proj=True,
                picks=picks,
                baseline=None,
                preload=True,
                verbose=False)
epochs.to_data_frame()
#y = epochs.events[:, -1]

#X = csp.fit_transform(1e6 *epochs.get_data(), y)  


In [ ]:
csp = CSP(n_components=4, reg='oas') 
data_e=data_EEG(multiclass=True, train=True)
y = data_e.events[:, -1]
X = csp.fit_transform( 1e6 *data_e.get_data(), y) 

In [ ]:
data_EEG(multiclass=False, train=False).get_data().shape


In [ ]:
# ... (Rest of the code: accuracy, training loop)
epochs = 50
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # 5-fold cross-validation




# Data Conversion (NumPy to PyTorch)


model = KAN(width=[X.shape[1], 2],  # Input features, 2 output classes
            grid=5,          
            k=3)             

# Ensure double-precision tensors
for param in model.parameters():
    param.data = param.data.double()

optimizer = LBFGS(model.parameters()) 



# --- MLP Model --- 
mlp_model = Sequential(
      Linear(X.shape[1], 8).double(),  # Input features 
      ReLU(),
      Linear(8, 2).double()  # 2 output classes 
)

# Ensure double-precision tensors
for param in mlp_model.parameters():
    param.data = param.data.double()

mlp_optimizer = LBFGS(mlp_model.parameters())  

for fold, (train_index, test_index) in enumerate(kf.split(X)):
    # Prepare data for this fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = KAN(width=[X_train.shape[1], 2],  # Input features, 2 output classes
            grid=5,          
            k=3)             

    # Ensure double-precision tensors
    for param in model.parameters():
        param.data = param.data.double()

    optimizer = LBFGS(model.parameters()) 



    # --- MLP Model --- 
    mlp_model = Sequential(
        Linear(X_train.shape[1], 8).double(),  # Input features 
        ReLU(),
        Linear(8, 2).double()  # 2 output classes 
    )

    # Ensure double-precision tensors
    for param in mlp_model.parameters():
        param.data = param.data.double()

    mlp_optimizer = LBFGS(mlp_model.parameters())  

# ... (Rest of the code: accuracy, training loop)
    epochs = 20

    for epoch in range(epochs):
        ### KAN Training
        def kan_closure():  
            optimizer.zero_grad()
            output = model(X_train)
            loss =  CrossEntropyLoss()(output, y_train)
            loss.backward()
            return loss
        optimizer.step(kan_closure)

        ### MLP Training
        def mlp_closure():
            mlp_optimizer.zero_grad()
            output = mlp_model(X_train)
            loss = CrossEntropyLoss()(output, y_train)  # Using standard loss
            loss.backward()
            return loss
        mlp_optimizer.step(mlp_closure)

        kan_train_acc, kan_train_f1, kan_train_kappa = calculate_metrics(model(X_train), y_train)
        kan_test_acc, kan_test_f1, kan_test_kappa = calculate_metrics(model(X_test), y_test) 

        mlp_train_acc, mlp_train_f1, mlp_train_kappa = calculate_metrics(mlp_model(X_train), y_train)
        mlp_test_acc, mlp_test_f1, mlp_test_kappa = calculate_metrics(mlp_model(X_test), y_test)

        print(f'KFOLD {fold}')
        print(f'Epoch {epoch+1}/{epochs} | '
            f'KAN Train Acc: {kan_train_acc:.3f} | KAN F1: {kan_train_f1:.3f} | KAN Kappa: {kan_train_kappa:.3f} | '
            f'MLP Train Acc: {mlp_train_acc:.3f} | MLP F1: {mlp_train_f1:.3f} | MLP Kappa: {mlp_train_kappa:.3f}') 